# Lesson 2: Protecting Routes with Middleware

# Protecting Routes with Middleware

Welcome to the next step in building your full-featured To-Do list application! Now that you have learned how to implement user authentication, it's crucial to ensure that only authenticated users can access specific parts of your application. In this unit, we will explore how to protect routes using middleware in Django.

## What You'll Learn
In this section, you'll discover how to implement middleware to secure your application routes. We'll cover the following key points:

- **Understanding Middleware**: Middleware acts as a bridge between the web server and the web application. It processes requests before they reach the view and can modify responses before sending them to the client.
- **Creating and Using Middleware**: You'll learn how to create custom middleware to check for user authentication.

### Example: Custom Middleware

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        # This returns the view function for the current route, e.g., 'www.example.com/login' resolves to 'login'
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if request.headers.get('Authorization') != 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)
```

### Code Breakdown:
1. **Unprotected Routes**: The `AuthMiddleware` class checks if the current route is in the `unprotected_routes` list. If it is, the middleware allows the request to proceed. This is useful for routes like login, register, and logout, which should be accessible to all users, regardless of authentication status.
2. **Authentication Check**: If the route is not in `unprotected_routes`, the middleware checks for a valid `Authorization` header. If missing or incorrect, it returns a `401 Unauthorized` response. (In real-world scenarios, tokens would be validated against a database rather than hardcoded.)
3. **Middleware Chain**: The middleware either calls the next middleware in the chain or proceeds to the view if there are no further middlewares.

### Applying Middleware in Django Settings

To protect your views with the custom middleware, add it to your Django settings:

```python
MIDDLEWARE = [
    # ... other middleware
    'myapp.middleware.AuthMiddleware',
]
```

## Returning Token in Login Response

To authenticate users, you need to return a token when they log in. This token can be used to authorize future requests. Here's an example of how to return a token in the login response:

```python
from django.http import JsonResponse
from django.contrib.auth import login, authenticate
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        # Validate username and password (not shown here)
        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the token in the response (for simplicity, we use a fixed token value)
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})
```

In this example, the token `'abc123'` is returned for demonstration purposes. In a real-world application, you would generate a unique token for each user and store it securely.

## Sending Token in Subsequent Requests

After logging in, you can send the token in the `Authorization` header for future requests:

```python
import requests

url = 'http://localhost:3000/protected-route'
headers = {'Authorization': 'Token abc123'}
response = requests.get(url, headers=headers)
```

In this example, a GET request is sent to `protected-route` with the token `'abc123'` in the `Authorization` header. This token is validated by the `AuthMiddleware` we created earlier.

## Why It Matters

Middleware is an essential part of web security because it:
- **Adds an Extra Layer of Security**: It checks each request before it reaches the view, ensuring only authorized users can access restricted parts of your application.
- **Centralizes Control**: Instead of adding checks in every view, you can handle access control in one place, making your code cleaner and easier to maintain.
- **Enhances User Experience**: By ensuring users are appropriately authenticated, you provide a smoother and more secure experience, preventing unauthorized access seamlessly.

Securing your application with middleware not only protects sensitive data but also adds professionalism and reliability to your app. Ready to dive into practice? Let's get started!

## Protecting Routes with Middleware

To observe the behavior of the `AuthMiddleware` that restricts access to protected routes, follow the steps described below:

1. **Middleware Overview**: The `AuthMiddleware` checks if a request is targeting a protected route and validates the `Authorization` token in the headers. If the correct token isn't provided, it returns a `401 Access Denied` response.

2. **Code Overview**:
    - The middleware protects any route not listed in `unprotected_routes` (e.g., 'login', 'register', 'logout'). 
    - If the `Authorization` header does not contain a valid token (in this example, `'Token abc123'`), access is denied.

### Running the Code:
1. **Register User**:
    - A new user is registered by sending a POST request to the `/register/` endpoint.
    - The response will confirm whether the registration was successful.

2. **Login User**:
    - The user logs in, and upon successful authentication, the server returns a fixed CSRF token (`'abc123'`).
    - This token is used for authentication in protected routes.

3. **Access Protected Route with Valid Token**:
    - A GET request is sent to the `/protected/` route, including the correct token (`'abc123'`) in the `Authorization` header.
    - Since the token is valid, the response should return `200 OK` along with the message `'This is a protected route'`.

4. **Access Protected Route with Invalid Token**:
    - A similar GET request is sent to the `/protected/` route, but this time the token is invalid (`'Token invalid_token'`).
    - The middleware will block the request, returning a `401 Access Denied` response.

5. **Logout User**:
    - Finally, the user logs out, and a success message is returned upon successful logout.

### Sample Output:

1. **Register Response**:
    ```
    Register Response: 201 - {'message': 'User registered successfully'}
    ```

2. **Login Response**:
    ```
    Login Response: 200 - {'message': 'User logged in successfully', 'csrf_token': 'abc123'}
    ```

3. **Access Protected Route with Valid Token**:
    ```
    Protected Route Response: 200 - {'message': 'This is a protected route'}
    ```

4. **Access Protected Route with Invalid Token**:
    ```
    Protected Route Response: 401 - {'message': 'Access denied'}
    ```

5. **Logout Response**:
    ```
    Logout Response: 200 - {'message': 'User logged out successfully'}
    ```

### Conclusion:
The middleware effectively protects the `/protected/` route by checking for a valid `Authorization` token in the request headers. If the token is missing or incorrect, the middleware prevents access, ensuring that only authenticated users with the correct token can access the protected parts of the application.

Nice work on understanding how middleware works! Now, let's put your knowledge into action by running some code and observing its output.

In this task, we have new AuthMiddleware that restricts access to protected routes when the correct token is not provided. The middleware checks if the current route is protected and if the correct token is provided in the request headers. If the token is not provided or is incorrect, the middleware returns a 401 status code with the message "Access denied".

Run the code and see how the middleware restricts access to protected routes when the correct token is not provided.

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)
        
        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)



from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def protected_view(request):
    return JsonResponse({'message': 'This is a protected route'})



import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")



from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, protected_view

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('protected/', protected_view, name='protected_view'),
]

```

## Protect Important Routes with Middleware

Building on our understanding of middleware, it's time to make a small adjustment.

In the AuthMiddleware class, we have a list of unprotected routes that are allowed to be accessed without a token. We need to modify the logic, to make the dashboard route protected as well.

Can you analyze the code and make the necessary changes to the AuthMiddleware class to ensure that the dashboard route is protected and requires a valid token to access it?

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout', 'dashboard']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)

from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def dashboard(request):
    return JsonResponse({'message': 'This is a protected route'})

import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'dashboard/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'dashboard/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")

from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, dashboard

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('dashboard/', dashboard, name='dashboard'),
]```

To make the `dashboard` route protected in your Django app, you need to modify the `AuthMiddleware` class. The current logic mistakenly includes `'dashboard'` in the `unprotected_routes`, meaning that it is accessible without a token. To fix this, simply remove `'dashboard'` from the `unprotected_routes` list.

Here's the updated `AuthMiddleware`:

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # 'dashboard' has been removed from unprotected routes, making it protected
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)

        return self.get_response(request)
```

### Changes Made:
- **Removed `'dashboard'` from `unprotected_routes`:**
  Now, access to the `/dashboard/` route is protected and requires a valid token in the `Authorization` header.

### How it Works:
1. **Unprotected Routes:** Routes like `login`, `register`, and `logout` are accessible without authentication.
2. **Protected Routes:** Now, `/dashboard/` is protected. It will require the correct token in the request header to allow access.

### Testing:

1. **Accessing the Protected Dashboard Route:**
   With a valid token (`Token abc123`), the middleware will allow access to the `/dashboard/` route:
   ```python
   # Request the protected route with valid token
   headers = {
       'Authorization': f'Token {csrf_token}'
   }
   response = session.get(BASE_URL + 'dashboard/', headers=headers)
   print(f"Protected Route Response: {response.status_code} - {response.json()}")
   ```

2. **Accessing with an Invalid Token:**
   If the token is invalid, the middleware will return a `401 Access Denied`:
   ```python
   # Request the protected route with invalid token
   headers = {
       'Authorization': 'Token invalid_token'
   }
   response = session.get(BASE_URL + 'dashboard/', headers=headers)
   print(f"Protected Route Response: {response.status_code} - {response.json()}")
   ```

Now, your `/dashboard/` route is correctly protected, requiring a valid token for access.

## Fix the Middleware Bug

Great job understanding how middleware works! Now, let's identify and fix a bug in the AuthMiddleware class.

Currently, the middleware is meant to protect certain routes by checking for an authorization token in the request headers. We also have certain routes, that must be accessible without a token, such as the login, register, and logout routes - so that users can register, login, and logout.

However, something is wrong with the setup, when users try to register, they get an "Access denied" message. This is not the expected behavior.

Can you identify the issue and fix it so that users can register successfully?

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)
        
        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def protected_view(request):
    return JsonResponse({'message': 'This is a protected route'})


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, protected_view

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('protected/', protected_view, name='protected_view'),
]


```

The issue lies in how the `AuthMiddleware` class checks for unprotected routes. In Django, `resolve(request.path_info).url_name` returns the name of the resolved view, which corresponds to the `name` attribute in `urlpatterns`.

To fix the issue where users cannot register due to "Access denied," we need to ensure that routes like `register` are correctly identified as unprotected. Here’s how you can adjust the `AuthMiddleware` class:

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response
        self.unprotected_routes = ['register', 'login', 'logout']

    def __call__(self, request):
        current_route = resolve(request.path_info).url_name

        if current_route in self.unprotected_routes:
            return self.get_response(request)
        
        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        
        return self.get_response(request)
```

### Explanation:
1. **Initialization of `unprotected_routes`:** Define a list of route names (`url_name` values) that do not require authorization (`register`, `login`, `logout`).
  
2. **Checking current route:** Use `resolve(request.path_info).url_name` to get the name of the current route being accessed.

3. **Authorization check:** If the current route is in `unprotected_routes`, allow access without further checks.

4. **Return response:** If the current route requires authorization and no valid token (`'Token abc123'`) is found in the request headers, return an "Access denied" response with status code 401.

By making these adjustments, routes like `register` will now be accessible without requiring an authorization token, allowing users to register successfully.

## Protect Routes with Middleware

Great job so far! Now, let's implement part of our application where we protect certain routes using middleware.

We are going to complete the AuthMiddleware class that will check if the user is authenticated before accessing certain routes.

Follow the instructions in the myapp/middleware.py file to complete the implementation of the AuthMiddleware class.

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        # TODO: Check if current route is in unprotected routes

            # TODO: Immediately return the response if route is unprotected skipping the token check
        
        # TODO: Check if authorization token in headers matches the expected token 'Token abc123'
            
            # TODO: Return a JsonResponse with message 'Access denied' and status 401 if token is incorrect
            
        return self.get_response(request)

from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def protected_view(request):
    return JsonResponse({'message': 'This is a protected route'})


import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Request the protected route with valid token
headers = {
    'Authorization': f'Token {csrf_token}'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")

# Request the protected route with invalid token
headers = {
    'Authorization': f'Token invalid_token'
}
response = session.get(BASE_URL + 'protected/', headers=headers)
print(f"Protected Route Response: {response.status_code} - {response.json()}")


from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, protected_view

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('protected/', protected_view, name='protected_view'),
]



```

Let's implement the `AuthMiddleware` class as outlined in the instructions. The goal is to protect certain routes by requiring a valid authorization token, while allowing unrestricted access to the unprotected routes (`login`, `register`, and `logout`).

Here's the complete implementation:

```python
from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        # Define unprotected routes that don't require authorization
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        # Check if the current route is unprotected
        if current_route in unprotected_routes:
            # Allow access to unprotected routes without checking the token
            return self.get_response(request)

        # Check if the 'Authorization' token is present in the request headers
        authorization_token = request.headers.get('Authorization')
        if authorization_token != 'Token abc123':
            # If the token is missing or incorrect, deny access
            return JsonResponse({'message': 'Access denied'}, status=401)

        # If everything is valid, proceed to the next middleware/view
        return self.get_response(request)
```

### Explanation of the Implementation:
1. **Unprotected Routes**: The list `unprotected_routes` contains routes that should not require an authorization token, such as `login`, `register`, and `logout`.
   
2. **Route Check**: The current route is determined using `resolve(request.path_info).url_name`, which extracts the `url_name` from the current request. If the route is in `unprotected_routes`, the middleware allows the request to pass through without checking for a token.

3. **Authorization Token Check**: For all other routes (protected routes), the middleware checks if the `Authorization` token in the request headers matches `'Token abc123'`. If the token is invalid or missing, a `JsonResponse` with a 401 status code is returned, denying access.

4. **Allow Valid Requests**: If the token is valid, the request is passed to the next middleware or view using `self.get_response(request)`.

### Usage:
- The middleware will now protect routes that are not included in the `unprotected_routes` list.
- When accessing a protected route, the client must provide a valid token (e.g., `'Authorization': 'Token abc123'`) in the headers.
- Unprotected routes (`login`, `register`, and `logout`) can be accessed without any token.

This implementation ensures that only authorized users can access certain protected routes, while users can still freely access the login, registration, and logout endpoints.

## Sending Requests with Token

Nice work on understanding how middleware works!

In this task, we will focus on how to send requests to the server to make sure that we provide all the necessary information to access the protected routes.

Follow the instructions in the send_request.py file to complete the code.

```python
import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# TODO: Construct the headers with the CSRF token and send a request to the protected route
    # Note - the key for the CSRF token in the headers should be 'Authorization'
    # and the value should be 'Token <csrf_token>'

# TODO: Send a request to the 'protected/' route with the headers

print(f"Protected Route Response: {response.status_code} - {response.json()}")


from django.http import JsonResponse
from django.urls import resolve

class AuthMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request):
        unprotected_routes = ['login', 'register', 'logout']
        current_route = resolve(request.path_info).url_name

        if current_route in unprotected_routes:
            return self.get_response(request)

        if not request.headers.get('Authorization') == 'Token abc123':
            return JsonResponse({'message': 'Access denied'}, status=401)
        return self.get_response(request)


from django.http import JsonResponse
from django.contrib.auth.decorators import login_required
from django.contrib.auth.models import User
from django.contrib.auth import login, authenticate, logout
from django.views.decorators.csrf import csrf_exempt
from django.middleware.csrf import get_token

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            # Return the CSRF token in the response. For simplicity, we are using a fixed token value.
            return JsonResponse({'message': 'User logged in successfully', 'csrf_token': 'abc123'})

        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)

@login_required
def protected_view(request):
    return JsonResponse({'message': 'This is a protected route'})


from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout, protected_view

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
    path('protected/', protected_view, name='protected_view'),
]


```

To complete the `send_request.py` file, the task involves constructing the correct headers with the CSRF token and sending a request to the protected route.

Here’s how to implement it:

```python
import requests

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

session = requests.Session()
username = "user_test"
email = "user@example.com"
password = "password"

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")
csrf_token = login_response.json().get('csrf_token')

# Construct the headers with the CSRF token and send a request to the protected route
headers = {
    'Authorization': f'Token {csrf_token}'  # Using the CSRF token obtained from login
}

# Send a GET request to the 'protected/' route with the headers
response = session.get(BASE_URL + 'protected/', headers=headers)

# Output the result of accessing the protected route
print(f"Protected Route Response: {response.status_code} - {response.json()}")
```

### Explanation:
1. **Login and Registration**: The user is first registered and logged in to retrieve a valid `csrf_token`.
   
2. **Constructing Headers**: Once the `csrf_token` is obtained from the login response, we construct the headers for the subsequent request to the protected route. The `Authorization` header is set to `Token <csrf_token>`.

3. **Sending Request to Protected Route**: A GET request is sent to the `protected/` route with the constructed headers. If the token is valid, the response will confirm access to the protected route.

4. **Printing the Response**: Finally, the response from the protected route is printed, showing whether access was granted or denied based on the token.

### Expected Behavior:
- **Successful access** to the protected route when a valid token (`Token abc123`) is provided.
- **Access denied** when an invalid or missing token is sent in the request. The server will respond with a `401 Unauthorized` status and a message like `'Access denied'`.

This should allow you to fully test the authentication and access control implemented with the middleware.